# DataPreprocessor: Класс для обработки и трансформации табличных данных

Этот notebook демонстрирует возможности класса **DataPreprocessor**, который выполняет базовые операции по очистке и трансформации табличных данных.

## Основные возможности:
1. **remove_missing()** - удаление столбцов с высокой долей пропусков и заполнение оставшихся
2. **encode_categorical()** - One-Hot кодирование категориальных переменных
3. **normalize_numeric()** - нормализация и стандартизация числовых признаков
4. **fit_transform()** - применение всех трансформаций в одной операции
5. **transform()** - применение сохранённого pipeline к новым данным

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')

# Установка стиля для графиков
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Библиотеки успешно загружены!")

In [ ]:
# Загрузка класса DataPreprocessor из модуля
import sys
sys.path.insert(0, r'g:\Testcase')

from data_preprocessor import DataPreprocessor

print("Класс DataPreprocessor успешно загружен!")

## Шаг 1: Загрузка и исследование датасета

Используем датасет **California Housing** из scikit-learn. Этот датасет содержит информацию о домах в Калифорнии с признаками, включая географические координаты, среднюю стоимость дома, количество комнат и т.д.

In [ ]:
# Загрузка датасета California Housing
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['Price'] = housing.target

# Информация о датасете
print("=" * 60)
print("ИНФОРМАЦИЯ О ДАТАСЕТЕ")
print("=" * 60)
print(f"\nФорма датасета: {df.shape}")
print(f"\nПервые 5 строк:")
print(df.head())
print(f"\nТипы данных:")
print(df.dtypes)
print(f"\nОсновная статистика:")
print(df.describe())

In [ ]:
# Добавление пропусков для демонстрации очистки данных
np.random.seed(42)

# Добавляем случайные пропуски
df_with_missing = df.copy()

# Столбец с 60% пропусков (будет удалён)
missing_indices = np.random.choice(len(df_with_missing), size=int(len(df_with_missing) * 0.6), replace=False)
df_with_missing.loc[missing_indices, 'Latitude'] = np.nan

# Столбец с 20% пропусков (будет заполнен)
missing_indices = np.random.choice(len(df_with_missing), size=int(len(df_with_missing) * 0.2), replace=False)
df_with_missing.loc[missing_indices, 'Longitude'] = np.nan

print("\nДатасет с добавленными пропусками:")
print(f"Процент пропусков по столбцам:")
print((df_with_missing.isnull().sum() / len(df_with_missing) * 100).sort_values(ascending=False))

## Шаг 2: Основная демонстрация работы класса DataPreprocessor

Создаём экземпляр класса и применяем полный pipeline обработки данных с методом `fit_transform()`.

In [ ]:
# Создание экземпляра класса DataPreprocessor
preprocessor = DataPreprocessor(df_with_missing)

# Применение полного pipeline с методом fit_transform()
df_processed = preprocessor.fit_transform(
    remove_missing_threshold=0.5,  # Удалять столбцы с 50%+ пропусков
    fill_method='mean',             # Заполнять пропуски средним значением
    normalize_method='minmax'        # Использовать Min-Max нормализацию
)

In [ ]:
# Вывод результатов преобразования
print("\n" + "=" * 60)
print("РЕЗУЛЬТАТЫ ПРЕОБРАЗОВАНИЯ")
print("=" * 60)

print(f"\nФорма обработанного датасета: {df_processed.shape}")
print(f"\nПервые 5 строк обработанного датасета:")
print(df_processed.head())
print(f"\nОсновная статистика после обработки:")
print(df_processed.describe())

## Шаг 3: Информация о применённых преобразованиях

Получаем подробную информацию об операциях, выполненных при обработке данных.

In [ ]:
# Получение информации о преобразованиях
preprocessing_info = preprocessor.get_preprocessing_info()

print("\n" + "=" * 60)
print("ИНФОРМАЦИЯ О ПРЕОБРАЗОВАНИЯХ")
print("=" * 60)

print(f"\n1. ИСХОДНЫЕ СТОЛБЦЫ ({len(preprocessing_info['original_columns'])} шт.):")
print(preprocessing_info['original_columns'])

print(f"\n2. УДАЛЁННЫЕ СТОЛБЦЫ ({len(preprocessing_info['removed_columns'])} шт.):")
print(preprocessing_info['removed_columns'] if preprocessing_info['removed_columns'] else "Нет")

print(f"\n3. КАТЕГОРИАЛЬНЫЕ СТОЛБЦЫ ({len(preprocessing_info['categorical_columns'])} шт.):")
print(preprocessing_info['categorical_columns'] if preprocessing_info['categorical_columns'] else "Нет")

print(f"\n4. ЧИСЛОВЫЕ СТОЛБЦЫ ({len(preprocessing_info['numeric_columns'])} шт.):")
print(preprocessing_info['numeric_columns'])

print(f"\n5. МЕТОД НОРМАЛИЗАЦИИ: {preprocessing_info['scaler_method']}")

print(f"\n6. ЗНАЧЕНИЯ ДЛЯ ЗАПОЛНЕНИЯ ПРОПУСКОВ:")
for col, val in preprocessing_info['fill_values'].items():
    print(f"   {col}: {val:.4f}")

print(f"\n7. ИТОГОВАЯ ФОРМА ДАТАСЕТА: {preprocessing_info['data_shape']}")

## Шаг 4: Сравнение методов нормализации

Демонстрация работы двух методов нормализации: Min-Max и стандартизация.

In [ ]:
# Применение Min-Max нормализации
preprocessor_minmax = DataPreprocessor(df_with_missing)
df_minmax = preprocessor_minmax.fit_transform(
    remove_missing_threshold=0.5,
    fill_method='mean',
    normalize_method='minmax'
)

print("\nОбработка с методом Min-Max нормализации завершена!")
print(f"Форма датасета: {df_minmax.shape}")

In [ ]:
# Применение стандартизации (Z-score normalization)
preprocessor_std = DataPreprocessor(df_with_missing)
df_std = preprocessor_std.fit_transform(
    remove_missing_threshold=0.5,
    fill_method='mean',
    normalize_method='std'
)

print("\nОбработка с методом стандартизации завершена!")
print(f"Форма датасета: {df_std.shape}")

## Шаг 5: Визуализация результатов нормализации

Сравнение распределений признаков при разных методах нормализации.

In [ ]:
# Выбираем несколько числовых столбцов для сравнения
numeric_cols_to_plot = ['MedInc', 'HouseAge', 'AveRooms']

fig, axes = plt.subplots(len(numeric_cols_to_plot), 3, figsize=(15, 10))
fig.suptitle('Сравнение методов нормализации', fontsize=16, fontweight='bold')

for idx, col in enumerate(numeric_cols_to_plot):
    # Исходные данные (с заполненными пропусками)
    preprocessor_temp = DataPreprocessor(df_with_missing)
    preprocessor_temp.remove_missing(threshold=0.5, fill_method='mean')
    
    axes[idx, 0].hist(preprocessor_temp.df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[idx, 0].set_title(f'{col}\n(Исходные данные)', fontweight='bold')
    axes[idx, 0].set_ylabel('Частота')
    
    # Min-Max нормализация
    axes[idx, 1].hist(df_minmax[col], bins=30, edgecolor='black', alpha=0.7, color='orange')
    axes[idx, 1].set_title(f'{col}\n(Min-Max нормализация)', fontweight='bold')
    
    # Стандартизация
    axes[idx, 2].hist(df_std[col], bins=30, edgecolor='black', alpha=0.7, color='green')
    axes[idx, 2].set_title(f'{col}\n(Стандартизация)', fontweight='bold')

plt.tight_layout()
plt.show()

print("Визуализация завершена!")

## Шаг 6: Применение сохранённого pipeline к новым данным

Демонстрация метода `transform()` для применения того же набора преобразований к новому датасету (например, тестовым данным).

In [ ]:
# Разделение данных на тренировочный и тестовый наборы
train_data = df_with_missing.iloc[:int(len(df_with_missing) * 0.8)]
test_data = df_with_missing.iloc[int(len(df_with_missing) * 0.8):]

print(f"Размер тренировочного набора: {train_data.shape}")
print(f"Размер тестового набора: {test_data.shape}")

# Создание и обучение preprocessor на тренировочных данных
train_preprocessor = DataPreprocessor(train_data)
train_processed = train_preprocessor.fit_transform(
    remove_missing_threshold=0.5,
    fill_method='mean',
    normalize_method='minmax'
)

print(f"\nЗначение Price (средняя стоимость) после нормализации:")
print(f"Min: {train_processed['Price'].min():.4f}, Max: {train_processed['Price'].max():.4f}")
print(f"Mean: {train_processed['Price'].mean():.4f}, Median: {train_processed['Price'].median():.4f}")

In [ ]:
# Применение сохранённого pipeline к тестовым данным
test_processed = train_preprocessor.transform(test_data)

print(f"\nПрименение pipeline к тестовым данным:")
print(f"Форма тестового набора до обработки: {test_data.shape}")
print(f"Форма тестового набора после обработки: {test_processed.shape}")

print(f"\nПервые 5 строк обработанного тестового набора:")
print(test_processed.head())

print(f"\nСравнение статистики тестового набора (должны быть в пределах [0, 1] для Min-Max):")
print(test_processed.describe())

## Шаг 7: Обработка ошибок и валидация параметров

Демонстрация работы обработки ошибок при неправильных входных параметрах.

In [ ]:
# Демонстрация обработки ошибок
print("=" * 60)
print("ДЕМОНСТРАЦИЯ ОБРАБОТКИ ОШИБОК")
print("=" * 60)

# Ошибка 1: Передача не-DataFrame
print("\n1. Попытка создать DataPreprocessor с не-DataFrame объектом:")
try:
    bad_preprocessor = DataPreprocessor([1, 2, 3])
except TypeError as e:
    print(f"   ✓ Перехвачена ошибка: {e}")

# Ошибка 2: Пустой DataFrame
print("\n2. Попытка создать DataPreprocessor с пустым DataFrame:")
try:
    empty_df = pd.DataFrame()
    bad_preprocessor = DataPreprocessor(empty_df)
except ValueError as e:
    print(f"   ✓ Перехвачена ошибка: {e}")

# Ошибка 3: Неправильный threshold
print("\n3. Попытка использовать threshold вне диапазона [0, 1]:")
try:
    preprocessor = DataPreprocessor(df_with_missing)
    preprocessor.remove_missing(threshold=1.5)
except ValueError as e:
    print(f"   ✓ Перехвачена ошибка: {e}")

# Ошибка 4: Неправильный метод нормализации
print("\n4. Попытка использовать неправильный метод нормализации:")
try:
    preprocessor = DataPreprocessor(df_with_missing)
    preprocessor.normalize_numeric(method='invalid_method')
except ValueError as e:
    print(f"   ✓ Перехвачена ошибка: {e}")

# Ошибка 5: Неправильный fill_method
print("\n5. Попытка использовать неправильный метод заполнения:")
try:
    preprocessor = DataPreprocessor(df_with_missing)
    preprocessor.remove_missing(fill_method='invalid')
except ValueError as e:
    print(f"   ✓ Перехвачена ошибка: {e}")

print("\n" + "=" * 60)
print("Все ошибки успешно обработаны!")

## Шаг 8: Воздействие порога пропусков на результат

Сравнение результатов при разных значениях threshold для удаления столбцов с пропусками.

In [ ]:
# Тестирование разных значений threshold
thresholds = [0.3, 0.5, 0.7, 0.9]

print("=" * 60)
print("ВОЗДЕЙСТВИЕ THRESHOLD НА КОЛИЧЕСТВО УДАЛЁННЫХ СТОЛБЦОВ")
print("=" * 60)

results = []

for threshold in thresholds:
    try:
        preprocessor = DataPreprocessor(df_with_missing)
        preprocessor.remove_missing(threshold=threshold, fill_method='mean')
        
        removed_count = len(preprocessor.removed_columns)
        remaining_count = len(preprocessor.df.columns)
        
        results.append({
            'Threshold': threshold,
            'Удалено столбцов': removed_count,
            'Осталось столбцов': remaining_count,
            'Удалённые': ', '.join(preprocessor.removed_columns) if preprocessor.removed_columns else 'Нет'
        })
    except Exception as e:
        print(f"Ошибка при threshold={threshold}: {e}")

# Вывод результатов в виде таблицы
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

## Выводы

### Основные возможности класса DataPreprocessor:

1. **Удаление столбцов с пропусками** (`remove_missing`)
   - Автоматически идентифицирует и удаляет столбцы с высокой долей пропусков
   - Заполняет оставшиеся пропуски средним значением, медианой или модой
   - Параметр `threshold` позволяет гибко управлять порогом удаления

2. **One-Hot кодирование** (`encode_categorical`)
   - Автоматически преобразует категориальные (строковые) переменные в бинарные признаки
   - Сохраняет информацию о всех созданных новых столбцах

3. **Нормализация признаков** (`normalize_numeric`)
   - Поддерживает Min-Max нормализацию (масштабирование на [0, 1])
   - Поддерживает стандартизацию (Z-score нормализация)
   - Применяется ко всем числовым столбцам

4. **Комплексный pipeline** (`fit_transform`)
   - Последовательно применяет все три преобразования
   - Сохраняет параметры для применения к новым данным

5. **Переиспользуемость** (`transform`)
   - Позволяет применять одинаковые преобразования к тестовым данным
   - Гарантирует консистентность между тренировочным и тестовым наборами

### Применение:
- Подготовка данных для машинного обучения
- Нормализация признаков перед использованием алгоритмов, чувствительных к масштабу
- Обработка пропущенных значений в реальных датасетах